In [34]:
import numpy as np # importing numpy for numerical computation
from sklearn.datasets import load_boston # here we are using sklearn's boston dataset
from sklearn.metrics import mean_squared_error

In [35]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variabl

In [36]:
x.shape

(506, 13)

In [37]:
def generating_samples(input_data, target_data):
    sampled_index=np.random.choice(len(input_data),303,replace=False) #sample index
    rep_index=np.random.choice(sampled_index,203)#replicate index
    k=np.random.randint(3,13,1) #select a column value between 3 and 13
    select_col=np.random.randint(3,13,k) #select k random columns
    sample_data=input_data[sampled_index[:,None],select_col]
    target_sample=target_data[sampled_index]
    rep_sample_data=input_data[rep_index[:,None],select_col]
    target_rep_data=target_data[rep_index]
    final_sample_data=np.vstack((sample_data,rep_sample_data))
    final_target_data=np.vstack((target_sample.reshape(-1,1),target_rep_data.reshape(-1,1)))
    return final_sample_data.tolist(),final_target_data.tolist(),sampled_index.tolist(),select_col.tolist()
    
    

In [38]:
def grader_samples(a,b,c,d):
    length = (len(a)==506  and len(b)==506)
    sampled = (len(a)-len(set([str(i) for i in a]))==203)
    rows_length = (len(c)==303)
    column_length= (len(d)>=3)
    assert(length and sampled and rows_length and column_length)
    return True
list_input_data=[]
list_output_data=[]
list_selected_row=[]
list_selected_columns=[]
for i in range(0,30):
    a,b,c,d = generating_samples(x, y)
    list_input_data.append(a)
    list_output_data.append(b)
    list_selected_row.append(c)
    list_selected_columns.append(d)
grader_samples(a,b,c,d)


True

In [39]:
def grader_30(a):
    assert(len(a)==30 and len(a[0])==506)
    return True
grader_30(list_input_data)

True

In [40]:
from sklearn.tree import DecisionTreeRegressor #training 30 models on 30 samples
list_of_all_models=[]
for i in range(30):
    model=DecisionTreeRegressor(max_depth=None)
    model.fit(list_input_data[i],list_output_data[i])
    list_of_all_models.append(model)


In [41]:
def mse(x,y,list_of_all_models,list_selected_columns): #calculating mse value
    y_predict=[] #
    for i in x:
        y_p=[]
        for k in range(len(list_of_all_models)):
            y_p.append(list_of_all_models[k].predict(i[list_selected_columns[k]].reshape(1,-1)))
        y_predict.append(np.sum(y_p)/30)
    me=0
    for k in range(len(y)):
        me=me+(y[k]-y_predict[k])**2
    return me/len(x)

In [42]:
def oob_score(x,y,list_of_all_models,list_selected_columns,list_selected_row): #calculating oob score value
    y_pred=[]
    for i in range(len(x)):
        count=0
        y_q=[]
        for j in range(len(list_of_all_models)):
            if i not in list_selected_row[j]:
                count+=1
                y_q.append(list_of_all_models[j].predict(x[i][list_selected_columns[j]].reshape(1,-1)))
        y_pred.append(np.sum(y_q)/count)        
    oob=0
    for i in range(len(y)):
        oob=oob+(y[i]-y_pred[i])**2
    return (oob/len(x))
    

In [43]:
o=oob_score(x,y,list_of_all_models,list_selected_columns,list_selected_row)
m=mse(x,y,list_of_all_models,list_selected_columns)
print("oob score :",o)
print("mse score :",m)


oob score : 14.326215057586893
mse score : 2.567669698302824


In [44]:
ob=[]
ms=[]
for j in range(35):
    list_input_data_2=[]
    list_output_data_2=[]
    list_selected_row_2=[]
    list_selected_columns_2=[]
    for i in range(0,30):
        a,b,c,d = generating_samples(x, y)
        list_input_data_2.append(a)
        list_output_data_2.append(b)
        list_selected_row_2.append(c)
        list_selected_columns_2.append(d)
    list_of_all_models_2=[]
    for i in range(30):
        model=DecisionTreeRegressor(max_depth=None)
        model.fit(list_input_data_2[i],list_output_data_2[i])
        list_of_all_models_2.append(model)
    obx=(oob_score(x,y,list_of_all_models_2,list_selected_columns_2,list_selected_row_2))
    mx=(mse(x,y,list_of_all_models_2,list_selected_columns_2))
    ob.append(obx)
    ms.append(mx)

In [45]:
import numpy as np #confidence interval
import scipy.stats 
def confidence_interval(data, confidence=0.95):#https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

q,w,e=confidence_interval(np.array(ob),confidence=0.95)
print("oob C.I :[",w,e,"]")
a,b,c=confidence_interval(np.array(ms),confidence=0.95)
print("mse C.I :[",b,c,"]")

oob C.I :[ 14.971249526565874 15.758424014573542 ]
mse C.I :[ 2.5911626392301197 2.8245198983694633 ]


In [50]:
x_q=np.array([0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60]) #house price
y_pe=[]
y_pret=[]
for k in range(len(list_of_all_models)):
    y_pe.append(list_of_all_models[k].predict(x_q[list_selected_columns[k]].reshape(1,-1)))
y_pret.append(np.sum(y_pe)/30)
print("House price :",y_pret)

House price : [20.869999999999997]


Task 1:
    step 1: Performing row sampling and column sampling and due to this each model 
    will learn  different features.
    step 2: models will be low bias and high variance .training models on the above data and finding MSE
    step 3: datapoints left out of  a certain sample are trained as cv and finding oob score
Task 2:
    we get 35 mse and oob values and  finding their confidence interval of their means
Task 3:
    predicting value of house price
    